Week 7 Notebook: Optimizing Other Objectives
===============================================================

This week, we will look at optimizing multiple objectives simultaneously. In particular, we will look at pivoting with adversarial neural networks {cite:p}`Louppe:2016ylz,ganin2014unsupervised,Sirunyan:2019nfw`.

We will borrow the implementation from: <https://github.com/glouppe/paper-learning-to-pivot>

In [ ]:
import tensorflow.keras as keras
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import uproot
from tqdm.notebook import tqdm

In [ ]:
import yaml

with open('definitions.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    definitions = yaml.load(file, Loader=yaml.FullLoader)
    
features = definitions['features']
spectators = definitions['spectators']
labels = definitions['labels']

nfeatures = definitions['nfeatures']
nspectators = definitions['nspectators']
nlabels = definitions['nlabels']
ntracks = definitions['ntracks']

## Define discriminator, regression, and combined adversarial models
The combined loss function is $$L = L_\mathrm{class} - \lambda L_\mathrm{reg}$$

- $L_\mathrm{class}$ is the loss function for the classification part (categorical cross entropy)
- $L_\mathrm{reg}$ is the loss function for the adversarial part (in this case a regression)
- $\lambda$ is a hyperparamter that controls how important the adversarial part of the loss is compared to the classification part, which we nominally set to 1

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Concatenate, GlobalAveragePooling1D
import tensorflow.keras.backend as K

# define Deep Sets model with Dense Keras layer
inputs = Input(shape=(ntracks, nfeatures,), name='input')  
x = BatchNormalization(name='bn_1')(inputs)
x = Dense(64, name='dense_1', activation='relu')(x)
x = Dense(32, name='dense_2', activation='relu')(x)
x = Dense(32, name='dense_3', activation='relu')(x)
# sum over tracks
x = GlobalAveragePooling1D(name='pool_1')(x)
x = Dense(100, name='dense_4', activation='relu')(x)
output = Dense(nlabels, name = 'output', activation='softmax')(x)
    
keras_model_disc = Model(inputs=inputs, outputs=output)
keras_model_disc.compile(optimizer='adam',
                        loss='categorical_crossentropy')

# regressor
x = Dense(100, name='dense_5', activation='relu')(keras_model_disc(inputs))
x = Dense(100, name='dense_6', activation='relu')(x)
output_reg = Dense(2, activation='linear', name='mass_pt_reg')(x)
                                                            

sgd_opt = keras.optimizers.SGD(momentum=0)
keras_model_reg = Model(inputs=inputs, outputs=output_reg)
keras_model_reg.compile(optimizer=sgd_opt,
                        loss='mse')

# combined model
lam = 1
keras_model_adv = Model(inputs=inputs, outputs=[keras_model_disc(inputs), keras_model_reg(inputs)])
keras_model_adv.compile(optimizer=sgd_opt, 
                        loss=['categorical_crossentropy', 'mse'],
                        loss_weights = [1, -lam])                              

print(keras_model_disc.summary())
print(keras_model_reg.summary())
print(keras_model_adv.summary())

## Load data

In [ ]:
from DataGenerator import DataGenerator
# load training and validation generators 
train_files = ['root://eospublic.cern.ch//eos/opendata/cms/datascience/HiggsToBBNtupleProducerTool/HiggsToBBNTuple_HiggsToBB_QCD_RunII_13TeV_MC/train/ntuple_merged_10.root']
val_files = ['root://eospublic.cern.ch//eos/opendata/cms/datascience/HiggsToBBNtupleProducerTool/HiggsToBBNTuple_HiggsToBB_QCD_RunII_13TeV_MC/train/ntuple_merged_11.root']


train_generator = DataGenerator(train_files, features, labels, spectators, batch_size=1024, n_dim=ntracks, 
                                remove_mass_pt_window=False, 
                                remove_unlabeled=True, max_entry=5000,
                                return_spectators=True, scale_mass_pt=[100., 10000.])

val_generator = DataGenerator(val_files, features, labels, spectators, batch_size=1024, n_dim=ntracks, 
                              remove_mass_pt_window=False, 
                              remove_unlabeled=True, max_entry=5000, 
                              return_spectators=True, scale_mass_pt=[100., 10000.])

## Pretrain discriminator and regressor models

In [ ]:
# pretrain discriminator
keras_model_disc.trainable = True
keras_model_disc.compile(optimizer='adam',
                         loss='categorical_crossentropy')
for n_epoch in tqdm(range(20)):
    for t in tqdm(train_generator, total=len(train_generator), leave=bool(n_epoch==19)):
        keras_model_disc.fit(t[0], t[1][0],verbose=0)        
        
# pretrain regressor
keras_model_reg.trainable = True
keras_model_disc.trainable = False
keras_model_reg.compile(optimizer=sgd_opt, loss='mse')
for n_epoch in tqdm(range(20)):
    for t in tqdm(train_generator, total=len(train_generator), leave=bool(n_epoch==19)):
        keras_model_reg.fit(t[0], t[1][1], verbose=0)                 

## Main training loop

During the main training loop, we do two things:
1. Train the discriminator model with the combined loss function $$L = L_\mathrm{class} - \lambda L_\mathrm{reg}$$
1. Train the regression model to learn the mass from with the standard MSE loss function $$L_\mathrm{reg}$$

In [ ]:
# alternate training discriminator and regressor                    
for n_epoch in tqdm(range(40)):
    for t in tqdm(train_generator, total=len(train_generator), leave=bool(n_epoch==39)):
        # train discriminator
        keras_model_reg.trainable = False
        keras_model_disc.trainable = True
        keras_model_adv.compile(optimizer=sgd_opt, 
                        loss=['categorical_crossentropy', 'mse'],
                        loss_weights=[1, -lam])    
        keras_model_adv.fit(t[0], t[1], verbose=0)

        # train regressor
        keras_model_reg.trainable = True
        keras_model_disc.trainable = False
        keras_model_reg.compile(optimizer=sgd_opt, loss='mse')
        keras_model_reg.fit(t[0], t[1][1],verbose=0)
keras_model_adv.save_weights('keras_model_adv_best.h5')

## Test

In [ ]:
# load testing file
test_files = ['root://eospublic.cern.ch//eos/opendata/cms/datascience/HiggsToBBNtupleProducerTool/HiggsToBBNTuple_HiggsToBB_QCD_RunII_13TeV_MC/test/ntuple_merged_0.root']
test_generator = DataGenerator(test_files, features, labels, spectators, batch_size=8192, n_dim=ntracks, 
                               remove_mass_pt_window=True, 
                               remove_unlabeled=True,
                               return_spectators=True,
                               max_entry=200000) # basically, no maximum

In [ ]:
# run model inference on test data set
predict_array_adv = []
label_array_test = []
spec_array_test = []

for t in tqdm(test_generator, total=len(test_generator)):
    label_array_test.append(t[1][0])
    spec_array_test.append(t[1][1])
    predict_array_adv.append(keras_model_adv.predict(t[0])[0])
predict_array_adv = np.concatenate(predict_array_adv, axis=0)
label_array_test = np.concatenate(label_array_test, axis=0)
spec_array_test = np.concatenate(spec_array_test, axis=0)

In [ ]:
# create ROC curves
print(label_array_test.shape)
print(spec_array_test.shape)
print(predict_array_adv.shape)
fpr_adv, tpr_adv, threshold_adv = roc_curve(label_array_test[:,1], predict_array_adv[:,1])
    
# plot ROC curves
plt.figure()
plt.plot(tpr_adv, fpr_adv, lw=2.5, label="Adversarial, AUC = {:.1f}%".format(auc(fpr_adv,tpr_adv)*100))
plt.xlabel(r'True positive rate')
plt.ylabel(r'False positive rate')
plt.semilogy()
plt.ylim(0.001, 1)
plt.xlim(0, 1)
plt.grid(True)
plt.legend(loc='upper left')
plt.show()

In [ ]:
from utils import find_nearest

In [ ]:
plt.figure()
for wp in [1.0, 0.5, 0.3, 0.1, 0.05]:
    idx, val = find_nearest(fpr_adv, wp)
    plt.hist(spec_array_test[:,0], bins=np.linspace(40, 200, 21), 
             weights=label_array_test[:,0]*(predict_array_adv[:,1] > threshold_adv[idx]),
             alpha=0.4, density=True, label='QCD, {}% FPR cut'.format(int(wp*100)),linestyle='-')
plt.legend()
plt.xlabel(r'$m_{SD}$')
plt.ylabel(r'Normalized probability')
plt.xlim(40, 200)

plt.figure()
for wp in [1.0, 0.5, 0.3, 0.1, 0.05]:
    idx, val = find_nearest(fpr_adv, wp)
    plt.hist(spec_array_test[:,0], bins=np.linspace(40, 200, 21), 
             weights=label_array_test[:,1]*(predict_array_adv[:,1] > threshold_adv[idx]),
             alpha=0.4, density=True, label='H(bb), {}% FPR cut'.format(int(wp*100)),linestyle='-')
plt.legend()
plt.xlabel(r'$m_{SD}$')
plt.ylabel(r'Normalized probability')
plt.xlim(40, 200)
plt.show()

plt.figure()
plt.hist(predict_array_adv[:,1], bins = np.linspace(0, 1, 21), 
             weights=label_array_test[:,1]*0.1,
             alpha=0.4, linestyle='-', label='H(bb)')
plt.hist(predict_array_adv[:,1], bins = np.linspace(0, 1, 21), 
             weights=label_array_test[:,0],
             alpha=0.4, linestyle='-', label='QCD')
plt.legend()
plt.show()


plt.figure()
plt.hist(spec_array_test[:,0], bins = np.linspace(40, 200, 21), 
             weights = label_array_test[:,1]*0.1,
             alpha=0.4, linestyle='-', label='H(bb)')
plt.hist(spec_array_test[:,0], bins = np.linspace(40, 200, 21), 
             weights = label_array_test[:,0],
             alpha=0.4, linestyle='-', label='QCD')
plt.legend()
plt.show()